<a href="https://colab.research.google.com/github/Kellen-Sullivan/AI535FinalProject/blob/main/AI535FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

## Import required libraries

In [2]:
from ultralytics import YOLO
from IPython.display import Image

## Download Dataset from Roboflow

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fZeOJWIPyFVGlTI0Q5lj")
project = rf.workspace("kellens-workspace-ausjh").project("underwater-trash-segmentation-io1hv")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...


## Train Yolov11 model on TrashCan1.0 Dataset

In [7]:
# Load the seg model
model = YOLO("yolo11n-seg.pt")

In [ ]:
# Train model on custom dataset
results = model.train(task='segment', mode='train', data='/content/Underwater-Trash-Segmentation-2/data.yaml', epochs=50)

Ultralytics 8.4.18 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Underwater-Trash-Segmentation-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, 

## Examining Results

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Load images using opencv
img1 = cv2.imread(f"{train_path}/BoxP_curve.png")
img2 = cv2.imread(f"{train_path}/BoxR_curve.png")
img3 = cv2.imread(f"{train_path}/MaskP_curve.png")
img4 = cv2.imread(f"{train_path}/MaskR_curve.png")

# Concatenate the images horizontally
top_row = cv2.hconcat([img1, img2])
bottom_row = cv2.hconcat([img3, img4])

# Concatentate the two rows vertically
grid = cv2.vconcat([top_row, bottom_row])

cv2_imshow(grid)



## Confusion Matrix

In [ ]:
Image("/content/runs/segment/train/confusion_matrix.png", width = 800)

## Model Prediction on Validation Batch

In [ ]:
Image("/content/runs/segment/train/val_batch0_pred.jpg", width = 800)

In [ ]:
Image("/content/runs/segment/train/results.png", width = 800)

## Validate Fine-Tuned Model

In [ ]:
metrics = model.val()
print("Model Evaluation Metrics: ", metrics)

## Inference on Test Image Dataset

In [ ]:
# Predict on test images using trained model
import os

# Load trained model
model = YOLO("/content/best.pt")

# Specify the test folder path NOTE: WE ONLY HAVE TRAIN AND VAL RN
test_path = "/content/Underwater-Trash-Segmentation-2/valid/images"

# List all images in the "test" (actually valid rn) folder
image_files = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith('.jpg', '.jpeg', '.png')]

# Predict with each image
for img_path in image_files:
  results = model.predict(source = img_path, save = True, conf = 0.2)

## Plot Predictiosn on Test (VAL RN) Images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

pred_path = '/content/runs/segment/predict'

# List all images in the predict folder
pred_image_files = [os.path.join(pred_path, img) for img in os.listdir(pred_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Number of images per row
images_per_row = 5

# Calculate the number of rows needed
n_rows = len(pred_image_files) // images_per_row + int(len(pred_image_files) % images_per_row != 0)

# Set figure size
fig, axs = plt.subplots(n_rows, images_per_row, figsize=(15, 3 * n_rows))
axs = axs.flatten()

# Loop through each image and display it
for i, img_path in enumerate(pred_image_files):
  img = mpimg.imread(img_path)
  axs[i].imshow(img)
  axs[i].axis('off')
  axs[i].set_title(f"Image {i+1}")

# Hide any extra subplots (if the number of images is not a perfect multiple images_of_rows)
for j in range(i+1, len(axs)):
  axs[j].axis('off') # Hide unused axes

# Display all images
plt.tight_layout()
plt.show()